In [1]:
# Make a bunch of batch isochrones around all line features within a line feature class/shapefile
# End goal is to have polygon showing all areas within network-based buffer distance of 
# lines

# If not already running, get the docker ORS image running by typing the following into CMD prompt:
# docker container start ors-app
# FYI - you must start docker Desktop before running this in the CMD

import os

import arcpy
import geopandas as gpd
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

import line_based_isochrone_localapi as lbi
from time import perf_counter
import datetime as dt

in_api_file = r"C:\Users\dconly\GitRepos\GIS-tools\ORS\api2_DO_NOT_COMMIT.txt"

# NOTE - for each mode, you must ensure that you have correct graphs built--see folder in openrouteservice repo
# under ~/docker/graphs. If you need to change/update graphs, set "active" profile in ors-config.json
# then run docker container restart ors-app in cmd prompt
mode = "foot-walking"  # "driving-car", "foot-walking", "cycling-regular"
isoctype = "distance" # "time", "distance" 
travel_range_mins = 1.0 # enter time in minutes, distance in miles

lines_fc = r'I:\Projects\Darren\TrailsAnalysis\TrailsAnalysis.gdb\TEST_MorrisonCrkSample'  
isoch_pts_per_mile = 10 # how close together you want the isochrones' origin points to be along the project line
output_fgdb = r"I:\Projects\Darren\TrailsAnalysis\TrailsAnalysis.gdb" # file geodatabase where output isochrone FC will go

print("all modules and parameters loaded successfully.")




all modules and parameters loaded successfully.


In [2]:
# =================RUN SCRIPT ==========================
start_time = perf_counter()
tstamp_str = str(dt.datetime.now().strftime('%Y%m%d_%H%M'))

# temp_fc_out = os.path.join(arcpy.env.scratchGDB, "TEMP_isoch_undissolved")

mode_short = mode.split('-')[0]
out_fc_name = f"isoch_{mode_short}"
output_fc = os.path.join(output_fgdb, f"{out_fc_name}{tstamp_str}")

isoch_pt_interval = 5280 / isoch_pts_per_mile

temp_ws = "memory" # arcpy.env.scratchGDB
temp_fcname = "tempfc"
sref_wgs84 = arcpy.SpatialReference(4326) # ORS needs WGS84

# make master geodataframe for all lines' isochrones; you'll append to it as you go.
gdf_comb_isos = gpd.GeoDataFrame()

with arcpy.da.SearchCursor(lines_fc, "SHAPE@") as cur:
    i = 0
    for row in cur: # for each line feature in the set of input lines:
        
        # create a temp feature class for the line
        temp_fc_path = os.path.join(temp_ws, temp_fcname)
        arcpy.management.CreateFeatureclass(temp_ws, temp_fcname, geometry_type="POLYLINE",
                                           spatial_reference=sref_wgs84)

        linegeom = row[0] # geometry of the line we're making a temp fc from
        with arcpy.da.InsertCursor(temp_fc_path, ["SHAPE@"]) as inscur: # insert line into temp fc
            inscur.insertRow([linegeom])
            
            
        # make a geodataframe for the isochrone corresponding to that line
        line_iso = lbi.ORSIsochrone(api_file=in_api_file, isoc_type=isoctype,
                    range_mins_or_mi=travel_range_mins, trav_mode=mode, batch_size=1)

        # build geodataframe of the line feature's isochrone
        gdf_line_iso = line_iso.make_line_isochrone(in_line_fc=temp_fc_path, interval_feet=isoch_pt_interval,
                                output_file=None)
        
        # then append that line's isochrone to the master geodataframe
        # gdf_comb_isos = gdf_comb_isos.append(gdf_line_iso)
        
        temp_undissolved = os.path.join(arcpy.env.scratchGDB, "TEMP_isoch_undissolved")
        temp_dissolved = os.path.join(arcpy.env.scratchGDB, "TEMP_isoch_dissolved")
        
        sedf = pd.DataFrame.spatial.from_geodataframe(gdf_line_iso) # convert to geoapndas gdf into spatially enabled dataframe
        lbi.sedf_to_fc_workaround(sedf, temp_undissolved) # get point isos FC for the line
        
        if i == 0:
            arcpy.management.Dissolve(temp_undissolved, output_fc) # dissolve so only 1 iso feature per line, this will be start of the output FC
        else:
            arcpy.management.Dissolve(temp_undissolved, temp_dissolved) 
            arcpy.management.Append(temp_dissolved, output_fc) # append the iso poly to the FC of iso polys for all lines
        
        if i % 50 == 0: 
            print(f"{i} features from input layer processed...")
        i += 1

# Export the combined gdf to feature class
# In future, can have option to combine into single large isochrone
# sedf = pd.DataFrame.spatial.from_geodataframe(gdf_comb_isos)
# lbi.sedf_to_fc_workaround(sedf, temp_fc_out) #convert output to feature class

# arcpy.management.Dissolve(temp_fc_out, output_fc)
# arcpy.management.Delete(temp_fc_out)isoch_3mibike_longtrl_existing 

elapsed = round((perf_counter() - start_time) / 60, 2)
print(f"Made isochrone around all line segments in {lines_fc} in {elapsed} minutes. Output is {output_fc}.")


0 features from input layer processed...
Made isochrone around all line segments in I:\Projects\Darren\TrailsAnalysis\TrailsAnalysis.gdb\TEST_MorrisonCrkSample in 1.44 minutes. Output is I:\Projects\Darren\TrailsAnalysis\TrailsAnalysis.gdb\isoch_foot20220401_1630.
